In [1]:
import pandas as pd
import numpy as np
import os
from bokeh.io import output_notebook, show
output_notebook()
import bokeh as bh
from sklearn.preprocessing import LabelEncoder
from glob import glob
import re
from math import ceil
import xgboost as xgb
from sklearn.model_selection import GroupKFold,KFold
import random
import operator
from sklearn.metrics import f1_score, confusion_matrix
pd.options.display.max_columns = 1000

Loading BokehJS ...

In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
os.chdir('C:/Users/pavan.velaga/Downloads')

C:\Users\pavan.velaga\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
data = pd.read_csv( "CAX_MortgageModeling_Train.csv")
submission_raw = pd.read_csv("CAX_MortgageModeling_Test.csv")

In [56]:
submission_raw.shape

(15214, 31)

In [5]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    for n in ( 'Day', 'Dayofweek', 'Month',
              'Is_month_end', 'Is_month_start'):
        df[targ_pre+n] = getattr(fld.dt, n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9

In [6]:
def feature_creation(df):
    df['anl_housing_exp'] = df['INCOME'] * df['GDS']
    df['appriesed_property_value'] = df['MORTGAGE AMOUNT']/df['LTV']
    df['other_expenses'] = (df['INCOME']*df['TDS']) - df['anl_housing_exp']
    df['property_apprl'] = df['appriesed_property_value']/df['PROPERTY VALUE']
    df['n_years'] = df['AMORTIZATION']/df['TERM']
    df['rate_property_apprl_yr'] = df['property_apprl']/df['n_years']
    df['rate_property_apprl_trm'] = df['property_apprl']/df['TERM']
    df['rate_property_apprl_amr'] = df['property_apprl']/df['AMORTIZATION']
    df['dipo_income'] = df['INCOME'] - df['other_expenses'] - df['anl_housing_exp']
    df['']
    
    return(df)

In [32]:
data = feature_creation(data)
submission_raw = feature_creation(submission_raw)

In [33]:
data.head(10)

,Unique_ID,MORTGAGE NUMBER,PROPERTY VALUE,MORTGAGE PAYMENT,GDS,LTV,TDS,AMORTIZATION,MORTGAGE AMOUNT,RATE,MORTGAGE PURPOSE,PAYMENT FREQUENCY,PROPERTY TYPE,TERM,FSA,AGE RANGE,GENDER,INCOME,INCOME TYPE,NAICS CODE,CREDIT SCORE,RESULT,target,anl_housing_exp,appriesed_property_value,other_expenses,property_apprl,n_years,rate_property_apprl_yr,rate_property_apprl_trm,rate_property_apprl_amr,dipo_income
0,CAX_Train_301,301,3525000,15033,7.96,80.0,23.90,360,2820000.0,5.00,Purchase,Monthly,Single Detached,12,M5P,70 and over,Male,3860000,8,23,671,NOT FUNDED,1,30725600.0,35250.000000,61528400.0,0.010000,30.0,0.000333,0.000833,0.000028,-88394000.0
1,CAX_Train_15869,15869,3525000,13922,8.75,65.0,28.01,360,2820000.0,5.25,Purchase,Monthly,Single Detached,12,M5P,70 and over,Male,3860000,8,23,692,FUNDED,0,33775000.0,43384.615385,74343600.0,0.012308,30.0,0.000410,0.001026,0.000034,-104258600.0
2,CAX_Train_40049,40049,2600000,9709,6.88,65.0,30.69,360,1690000.0,4.75,Refinance,Monthly,Single Detached,12,M4R,30-34,Unknown,3430000,8,23,755,FUNDED,0,23598400.0,26000.000000,81668300.0,0.010000,30.0,0.000333,0.000833,0.000028,-101836700.0
3,CAX_Train_29403,29403,4300000,22149,7.12,75.0,17.06,300,3225000.0,5.75,Purchase,Monthly,High Rise,12,M4W,60-64,Male,2441000,8,53,823,NOT FUNDED,1,17379920.0,43000.000000,24263540.0,0.010000,25.0,0.000400,0.000833,0.000033,-39202460.0
4,CAX_Train_4672,4672,900000,3960,3.38,75.0,9.79,360,675000.0,5.00,Purchase,Monthly,Single Detached,12,L3P,45-49,Male,2400000,8,23,734,FUNDED,0,8112000.0,9000.000000,15384000.0,0.010000,30.0,0.000333,0.000833,0.000028,-21096000.0
5,CAX_Train_25731,25731,776000,3719,2.26,80.0,26.09,360,621176.0,4.75,Purchase,Monthly,Single Detached,12,L5M,35-39,Unknown,2234000,8,53,750,FUNDED,0,5048840.0,7764.700000,53236220.0,0.010006,30.0,0.000334,0.000834,0.000028,-56051060.0
6,CAX_Train_32089,32089,635000,3075,3.46,80.0,10.34,360,596000.0,3.75,Purchase,Monthly,Single Detached,12,L3S,70 and over,Male,2200000,8,52,818,FUNDED,0,7612000.0,7450.000000,15136000.0,0.011732,30.0,0.000391,0.000978,0.000033,-20548000.0
7,CAX_Train_21602,21602,625000,2044,0.75,75.0,31.93,360,375000.0,4.00,Purchase,Monthly,High Rise,12,M5R,50-54,Male,2125000,8,72,650,FUNDED,0,1593750.0,5000.000000,66257500.0,0.008000,30.0,0.000267,0.000667,0.000022,-65726250.0
8,CAX_Train_1583,1583,400000,1817,1.10,80.0,2.29,360,320000.0,3.75,Purchase,Monthly,Single Detached,12,L4N,45-49,Male,2089000,4,81,810,NOT FUNDED,1,2297900.0,4000.000000,2485910.0,0.010000,30.0,0.000333,0.000833,0.000028,-2694810.0
9,CAX_Train_15238,15238,5000000,22919,10.92,80.0,29.49,360,4000000.0,4.50,Purchase,Monthly,Single Detached,24,M5P,40-44,Female,1930000,8,23,636,FUNDED,0,21075600.0,50000.000000,35840100.0,0.010000,15.0,0.000667,0.000417,0.000028,-54985700.0


In [34]:
from sklearn.model_selection import train_test_split

In [49]:
data = data.reset_index()


In [50]:
features = ['PROPERTY VALUE', 'MORTGAGE PAYMENT',
       'GDS', 'LTV', 'TDS', 'AMORTIZATION', 'MORTGAGE AMOUNT', 'RATE',
       'MORTGAGE PURPOSE', 'PAYMENT FREQUENCY', 'PROPERTY TYPE', 'TERM', 'FSA',
       'AGE RANGE', 'GENDER', 'INCOME', 'INCOME TYPE', 'NAICS CODE','anl_housing_exp','appriesed_property_value'
            ,'property_apprl','other_expenses','rate_property_apprl_trm','dipo_income']
target = ['target']

In [51]:
X_train, X_test, y_train = data[features], submission_raw[features], data[target]

In [52]:
for f in features:
    if X_train[f].dtype=='object':
        print(f+"-"+str(X_train[f].dtype))
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))
        

MORTGAGE PURPOSE-object
PAYMENT FREQUENCY-object


C:\Users\pavan.velaga\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\pavan.velaga\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


PROPERTY TYPE-object
FSA-object
AGE RANGE-object
GENDER-object
NAICS CODE-object


In [53]:
def runLGB(train_X, train_y, test_X, test_y=None, test_X2=None, dep=8, seed=0, data_leaf=511, hessian_leaf=50):

    dtrain =  xgb.DMatrix(X_train[features], label=y_train)
    dtest = xgb.DMatrix(test_X[features],label=y_test)
    params = {'objective': 'binary:logistic',
          'booster': 'gbtree',
          'eval_metric': 'auc',
          'nthread': 4,
          'max_depth': 5,
          'subsample': 0.8,
          'min_child_weight': 1,
          "colsample_bytree": 0.9,
          'eta': 0.05,
          'verbose_eval': True,
          'silent':1,
          'seed': seed
          }
    watchlist = [(dtrain, 'train'), (dtest, 'val')]

    clf_xgb = xgb.train(params, dtrain, num_boost_round=100, verbose_eval=100, early_stopping_rounds=30, evals=watchlist)

    
    
    pred_test_y = clf_xgb.predict(dtest)
    pred_test_y2 = clf_xgb.predict(dtest_2)
    return clf_xgb, pred_test_y, pred_test_y2

In [54]:
val_auc = []
pred_test_full = 0
pred_train = np.zeros(X_train.shape[0])
n_splits = 3
kf = KFold(n_splits=n_splits, shuffle=True, random_state=99)
#gkf = KFold(n_splits=n_splits)
# dtest_2 = xgb.DMatrix(X_test)
#model_name = "xgb"
for dev_index, val_index in kf.split(X_train, y_train):
    dev_X, val_X = X_train.iloc[dev_index,:], X_train.iloc[val_index,:]
    dev_y, val_y = y_train[dev_index], y_train[val_index]
    pred_val = 0
    pred_test = 0
    n_models = 0.
    clf_xgb, pred_v, pred_t = runLGB(dev_X, dev_y, val_X, val_y, None, seed=2019)
    pred_val += pred_v
    pred_test += pred_t
    n_models += 1
    break
    

KeyError: '[    0     4     5 ... 45638 45639 45640] not in index'

In [46]:
    pred_val = 0
    pred_test = 0
    n_models = 0.

    model, loss, pred_v, pred_t = runLGB(dev_X, dev_y, val_X, val_y, test_X, seed=2019)
    pred_val += pred_v
    pred_test += pred_t
    n_models += 1
    
    model, loss, pred_v, pred_t = runLGB(dev_X, dev_y, val_X, val_y, test_X, data_leaf=450, hessian_leaf=30, seed=9873)
    pred_val += pred_v
    pred_test += pred_t
    n_models += 1
    
    model, loss, pred_v, pred_t = runLGB(dev_X, dev_y, val_X, val_y, test_X, data_leaf=600, hessian_leaf=70, seed=4568)
    pred_val += pred_v
    pred_test += pred_t
    n_models += 1
    
    pred_val /= n_models
    pred_test /= n_models
    
    loss = roc_auc_score(val_y, pred_val)
        
    pred_train[val_index] = pred_val
    pred_test_full += pred_test / n_splits
    cv_scores.append(loss)
#     break
print(np.mean(cv_scores))

NameError: name 'test_X' is not defined

In [ ]:
#num_rounds = 130
xgb_preds_lst = []
for i in ([0,99,999]):
    watchlist = [(dtrain, 'train'), (dtest, 'val')]
    params = {'objective': 'binary:logistic',
              'booster': 'gbtree',
              'eval_metric': 'auc',
              'nthread': 4,
              'max_depth': 5,
              'subsample': 0.8,
              'min_child_weight': 1,
              "colsample_bytree": 0.9,
              'eta': 0.05,
              'verbose_eval': True,
              'silent':1,
              'seed': i
              }

    clf_xgb = xgb.train(params, dtrain, num_boost_round=1000, verbose_eval=100, early_stopping_rounds=30, evals=watchlist)
    xgb_preds_lst.append(clf_xgb.predict(dtest))  

In [ ]:
val_preds= [(xgb_preds_lst[0][i]+xgb_preds_lst[1][i]+xgb_preds_lst[2][i])/3 for i in range(len(xgb_preds_lst[1]))]


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(clf_xgb,max_num_features=50, height=0.8, ax=ax)
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(clf_xgb,importance_type= "gain", max_num_features=50, height=0.8, ax=ax)
plt.show()


In [ ]:
importance = clf_xgb.get_score(importance_type='gain')
importance_df = pd.DataFrame(
    sorted(importance.items(), key=operator.itemgetter(1)), 
    columns=['feature','gain']
    )

In [ ]:
class MetricsMeter():    
    def __init__(self, y_true, y_pred, threshold):
        self.y_true = y_true        
        self.y_pred = y_pred        
        self.thresh = threshold            
    
    def fit(self):        
        metrics = {}        
        metrics['auc'] = roc_auc_score(self.y_true, self.y_pred)                
        self.y_pred = [1 if x > self.thresh else 0 for x in self.y_pred]        
        metrics['f1'] = f1_score(self.y_true, self.y_pred,average = 'macro')        
        tn, fp, fn, tp = confusion_matrix(self.y_true, self.y_pred).ravel()                
        metrics['sensitivity/recall'] = tp / (tp+fn)
        metrics['precision'] = tp / (tp+fp)
        metrics["accuracy"] = (tp + tn) / (tp + fp + fn + tn)
        metrics['specificity'] = tn / (tn+fp)
        return metrics

In [ ]:
dr_metrics = MetricsMeter(y_test, val_preds, i)
scores = dr_metrics.fit()    
scores['thresh'] = i    

In [ ]:
m = [] 
for i in np.linspace(0.01, 0.99, num=90):    
    dr_metrics = MetricsMeter(y_test, val_preds, i)
    scores = dr_metrics.fit()    
    scores['thresh'] = i    
    m.append(scores) 
m_build = pd.DataFrame(m)


# 

In [ ]:
a = m_build.sort_values('f1', ascending=False)

In [ ]:
a.head(10) #0.635282 #0.658185 #0.662413 #0.662483

In [ ]:
dsub = xgb.DMatrix(submission_raw[features])
submission_raw['probs'] = clf_xgb.predict(dsub)

In [ ]:
submission_raw['Result_Predicted'] = np.where(submission_raw['probs']>0.307303,'NOT FUNDED','FUNDED')

In [ ]:
inter_data = pd.concat([X_test,y_test],axis = 1) 
inter_data['preds'] = val_preds
 #inter_data = pd.concat([inter_data,np.array(val_preds)],axis = 1)

In [ ]:
final_sub = submission_raw[['Unique_ID','Result_Predicted']]

In [ ]:
final_sub['Result_Predicted'].value_counts()

In [ ]:
final_sub.to_csv('new_ft_sub8.csv',index=False)

In [ ]:
#inter_data.to_csv('inter_data_probs.csv',index=False)

In [ ]:
#inter_data

In [ ]:
submission_raw.head(10)